# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from time import time

from data import FESDDataset
from data import Frame, AugmentationParams
import json
import numpy as np

import cv2

from model import FESD, train
import copy

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F

from utils import AvgMeter, clip_gradient, get_scheduler

from tqdm import tqdm

num_gpus = torch.cuda.device_count()
print(f"Num cuda GPUs: {num_gpus}")
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

h:\FESD\FESDModel\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num cuda GPUs: 1


In [3]:
RECORDING_DIR = Path('H:/Recordings/')
CHECKPOINT_DIR = Path('checkpoints')

## Data Loading

Firstly we need to import all the recordings into the notebook.


In [4]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

In [5]:
batchsize = 10
train_size = 300

dataset = FESDDataset(RECORDING_DIR, train_size)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize, shuffle=True)

Recordings Found: 25
Total Frames: 7500


In [6]:
dataset.randomize_augmentation_params = False
dataset.reset_augmentation_params()
rgb, depth, pose_2d, errors = dataset[0]
dataset.frame.show()

print(rgb.shape, depth.shape, pose_2d.shape, errors.shape)

print("All Missing Joints")
print(len(errors[errors==1]))

print("All Wrong Joints")
print(pose_2d[:,errors==2])

(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
torch.Size([3, 300, 300]) torch.Size([1, 300, 300]) torch.Size([3, 25]) torch.Size([25])
All Missing Joints
5
All Wrong Joints
tensor([[ -1.4217,  -1.1519],
        [167.1556, 170.9957],
        [  0.2603,   0.2511]])


### Build Model

The model proposed by RD3D is based on resnet50 so we copy a pretrained resnet50 model

In [7]:
resnet = torchvision.models.resnet50(pretrained=True)

model = FESD()
print(model)

model = nn.DataParallel(model).cuda()

h:\FESD\FESDModel\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
h:\FESD\FESDModel\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FESD(
  (rgb_conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (rgb_conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (rgb_conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (rgb_conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (depth_conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (depth_conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (depth_conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (depth_conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (joint_conv1): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (joint_conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (joint_conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=41600, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_

## Train Model

In the following we define the training function and train a network on the training data.

In [8]:
# epoch number
epochs = 50
# optimizer
optim = 'sdg'
# learning rate
learning_rate = 0.000125
# learning rate scheduler. can be step, poly or cosine
lr_scheduler = 'cosine'
# warmup epoch
warmup_epoch = -1
# warmup multiplier
warmup_multiplier = 100
# for step scheduler. where to decay lr, can be a list
lr_decay_epochs = [120, 160, 200]
# for step scheduler. step size to decay lr
lr_decay_steps = 20 
# for step scheduler. decay rate for learning rate
lr_decay_rate = 0.1
# weight decay
weight_decay = 0.0001
# momentum for SGD
momentum = 0.9
# gradient clipping margin
clip = 0.5

In [9]:
n_data = len(train_loader.dataset)
CE = torch.nn.CrossEntropyLoss().cuda()

if optim == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
elif optim == 'adamW':
    optimizer = torch.optim.AdamW(model.parameters(), learning_rate, weight_decay=weight_decay)
elif optim == 'sdg':
    optimizer = torch.optim.SGD(model.parameters(), learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)

scheduler = get_scheduler(optimizer, len(train_loader), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch, warmup_multiplier)

In [10]:
dataset.augmentation_params

AugmentationParams(flip=False, crop=False, crop_random=False, crop_pad=0, gaussian=False, seed=-1)

In [11]:
torch.cuda.empty_cache()
dataset.randomize_augmentation_params = False
# routine
for epoch in range(1, epochs + 1):
    tic = time()
    train(train_loader, model, optimizer, CE, scheduler, clip, epoch, epochs)
    print(f'epoch {epoch}, total time {time() - tic:.2f}, learning_rate {optimizer.param_groups[0]["lr"]}')

    if (epoch) % 10 == 0:
        torch.save(model.state_dict(), os.path.join(opt.output_dir, f"RD3D_{epoch}_ckpt.pth"))

        print("checkpoint saved {}!".format(os.path.join(opt.output_dir, f"{epoch}_ckpt.pth")))
        
# torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"last_ckpt.pth"))
print(f"model saved {os.path.join(CHECKPOINT_DIR, f'last_ckpt.pth')}!")
checkpoint = os.path.join(opt.output_dir, f"last_ckpt.pth")

(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 480, 1)
torch.Size([1, 300, 300])
torch.Size([1, 300, 300])
(480, 480, 3) (480, 480, 1)
(480, 480, 3) (480, 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x23043200 and 41600x1024)